<a href="https://colab.research.google.com/github/krishnakumar51/Weaviate/blob/main/Weaviate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Weaviate

In [ ]:
!pip -q install langchain  weaviate-client

In [ ]:
!pip install openai

In [ ]:
import os


In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')


In [ ]:
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [ ]:
!pip install unstructured
!pip install "unstructured[pdfs]"

In [ ]:
WEAVIATE_API_KEY= "8Gaj809zkg4zDT6QVkHBokP5lwM9wC8hVA2p"
WEAVIATE_CLUSTER="https://example-belxsspv.weaviate.network"

In [ ]:
# data reading

In [ ]:
!pip install pypdf


In [ ]:
!mkdir pds

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("pds")
data = loader.load()
data

[Document(page_content='You Only Look Once:\nUniﬁed, Real-Time Object Detection\nJoseph Redmon∗, Santosh Divvala∗†, Ross Girshick¶, Ali Farhadi∗†\nUniversity of Washington∗, Allen Institute for AI†, Facebook AI Research¶\nhttp://pjreddie.com/yolo/\nAbstract\nWe present YOLO, a new approach to object detection.\nPrior work on object detection repurposes classiﬁers to per-\nform detection. Instead, we frame object detection as a re-\ngression problem to spatially separated bounding boxes and\nassociated class probabilities. A single neural network pre-\ndicts bounding boxes and class probabilities directly from\nfull images in one evaluation. Since the whole detection\npipeline is a single network, it can be optimized end-to-end\ndirectly on detection performance.\nOur uniﬁed architecture is extremely fast. Our base\nYOLO model processes images in real-time at 45 frames\nper second. A smaller version of the network, Fast YOLO,\nprocesses an astounding 155 frames per second while\nstill a

In [ ]:
# chucnking

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(data)

In [ ]:
# embedding

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [ ]:
# vector_db_Storage

In [ ]:
import weaviate
from langchain.vectorstores import Weaviate

#Connect to weaviate Cluster
auth_config = weaviate.auth.AuthApiKey(api_key = WEAVIATE_API_KEY)
WEAVIATE_URL = WEAVIATE_CLUSTER

client = weaviate.Client(
    url = WEAVIATE_URL,
    additional_headers = {"X-OpenAI-Api-key": OPENAI_API_KEY},
    auth_client_secret = auth_config,
    startup_period = 10
)

In [ ]:
client.is_ready()

True

In [ ]:
# define input structure
client.schema.delete_all()
client.schema.get()
schema = {
    "classes": [
        {
            "class": "Chatbot",
            "description": "Documents for chatbot",
            "vectorizer": "text2vec-openai",
            "moduleConfig": {"text2vec-openai": {"model": "ada", "type": "text"}},
            "properties": [
                {
                    "dataType": ["text"],
                    "description": "The content of the paragraph",
                    "moduleConfig": {
                        "text2vec-openai": {
                            "skip": False,
                            "vectorizePropertyName": False,
                        }
                    },
                    "name": "content",
                },
            ],
        },
    ]
}

client.schema.create(schema)
vectorstore = Weaviate(client, "Chatbot", "content", attributes=["source"])

In [ ]:
# load text into the vectorstore
text_meta_pair = [(doc.page_content, doc.metadata) for doc in docs]
texts, meta = list(zip(*text_meta_pair))
vectorstore.add_texts(texts, meta)

{'error': [{'message': 'update vector: connection to: OpenAI API failed with status: 429 error: Rate limit reached for text-embedding-ada-002 in organization org-IO3pL04Z6cxEGyxLzs31W7w5 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.'}]}
{'error': [{'message': 'update vector: connection to: OpenAI API failed with status: 429 error: Rate limit reached for text-embedding-ada-002 in organization org-IO3pL04Z6cxEGyxLzs31W7w5 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.'}]}
{'error': [{'message': 'update vector: connection to: OpenAI API fai

['b44ec16c-8c66-49f3-9c1f-6245f18c136d',
 '2d6e5458-7a49-4e2a-b02e-0867a8c0dfc1',
 'e066be35-fc4d-4dc5-a0fc-1304e4601e69',
 '9635c74b-6119-49fe-917a-acb3cd29914f',
 '3d8b6227-e33d-4789-bbd4-74e250d039cb',
 'd9a49fe2-2b0b-498a-9146-b596ed60f400',
 '922e67c0-c690-492c-9fd7-9c9879acbb41',
 'b7d9a685-b924-48ad-9968-66ec5f596081',
 'e9773e4f-852e-48e0-9256-3a432e275b4e',
 'f6fc9399-bead-4764-8101-e2e6d0485a27',
 'de53e25b-b042-4f09-b5a1-f1eb5fbd275e',
 '723644f5-5f90-42e0-abf7-3869b60f3ced',
 'eb7945e3-4f50-4ecd-8295-9eff03b5ac7c',
 'ddb9d9aa-c6e3-4dba-922e-bbc385530eed',
 '5b7fbb81-dd4e-4974-92ce-cd8bb52d4edf',
 'dc96887a-53ab-4491-9eca-f982287bd757',
 '14d11d86-0c59-445f-b007-f1ab838206ca',
 '81f07e8a-d84b-4a18-9056-06413ec84b66',
 '8dba0b30-fa93-4dcf-8377-76309bc857cf',
 'fc267f7a-4653-4f64-b18c-0f3b84a573aa',
 '08bdb32d-de31-4a21-a467-ba4315e47f87',
 '9396655c-9fb8-4dc3-b880-4233f6ba9d8b',
 'bd904ebe-4e41-4fa3-aef6-7de4b3185ec3',
 'c1fcc721-06c7-4972-8828-c518225e2dcc',
 '9ed4a2dc-e75c-

In [ ]:
# similarity measurement

In [ ]:
query = "what is a yolo?"

# retrieve text related to the query
docs = vectorstore.similarity_search(query, top_k=3)

In [ ]:
docs

[Document(page_content='making predictions. Unlike sliding window and region\nproposal-based techniques, YOLO sees the entire image\nduring training and test time so it implicitly encodes contex-\ntual information about classes as well as their appearance.\nFast R-CNN, a top detection method [14], mistakes back-\nground patches in an image for objects because it can’t see\nthe larger context. YOLO makes less than half the number\nof background errors compared to Fast R-CNN.\nThird, YOLO learns generalizable representations of ob-\njects. When trained on natural images and tested on art-\nwork, YOLO outperforms top detection methods like DPM\nand R-CNN by a wide margin. Since YOLO is highly gen-\neralizable it is less likely to break down when applied to\nnew domains or unexpected inputs.\nYOLO still lags behind state-of-the-art detection systems\nin accuracy. While it can quickly identify objects in im-\nages it struggles to precisely localize some objects, espe-\ncially small ones. We

In [ ]:
# questinonming through llm

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
# define chain
chain = load_qa_chain(
    OpenAI(),
    chain_type="stuff")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [ ]:
# create answer
chain.run(input_documents=docs, question=query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' YOLO is a state-of-the-art object detection system that is able to quickly identify objects in images and learn generalizable representations of objects. However, it still lags behind other detection systems in terms of accuracy.'